In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

####################################################################################################################################################################################

## [1] This section removes bootstrap values from the trees (global lineages 1-6) with R for input into SNPPar

####################################################################################################################################################################################

### Run this for global lineages 1, 2, 3 4A, 4B, 4C, 5 & 6

In [3]:
global_lineage = '2' #specify global lineage, will modify this tree

#### R code below (only has to be run once for each tree)

Needs to be run in an interactive session

In [4]:
tree_files_output = '/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/phylogenies/tree_output_files/phylogeny_lineage_' + global_lineage
newick_tree_file_path = '/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/phylogenies/tree_output_files/phylogeny_lineage_' + global_lineage + '/tree_lineage_' + global_lineage + '_iqtree_FINAL.treefile'
newick_tree_file_no_bootsrap_vals_path = '/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/phylogenies/tree_output_files/phylogeny_lineage_' + global_lineage + '/tree_lineage_' + global_lineage + '_iqtree_FINAL_no_bootstrap_vals.treefile'

#load R module
command_1 = "module load R/3.4.1-extra"

#start R
command_2 = "R"

#load tree library
command_3 = "library(ape)"

#change working dir to the one with the phylogeny
command_4 = "setwd(" + "\"" + tree_files_output + "\"" + ")"

#load and store tree generated by IQtree in variable
command_5 = "tree <- read.tree(" + "\"" + newick_tree_file_path + "\"" + ")"

#replace all of the bootstrap values in the tree with nothing
command_6 = "tree$node.label <- NULL"

#save new tree (with bootstrap value removed) in same directory as original tree
command_7 = "write.tree(tree, file =" + "\"" + newick_tree_file_no_bootsrap_vals_path + "\"" + ")"

for command_i in [ command_1 , command_2 , command_3 , command_4 , command_5 , command_6 , command_7 ]:
    print command_i

module load R/3.4.1-extra
R
library(ape)
setwd("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/phylogenies/tree_output_files/phylogeny_lineage_2")
tree <- read.tree("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/phylogenies/tree_output_files/phylogeny_lineage_2/tree_lineage_2_iqtree_FINAL.treefile")
tree$node.label <- NULL
write.tree(tree, file ="/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/phylogenies/tree_output_files/phylogeny_lineage_2/tree_lineage_2_iqtree_FINAL_no_bootstrap_vals.treefile")


####################################################################################################################################################################################

## [2] This section converts and stores genotypes for all isolates belonging to a specific global lineage (tree) into a SNP table

####################################################################################################################################################################################

### Run this for global lineages 1, 2, 3, 4A, 4B, 4C, 5 & 6

In [2]:
global_lineage = '2'

In [7]:
import os
import pandas as pd
import numpy as np
import sys

import Bio
from Bio.Alphabet import IUPAC
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import pairwise2
from Bio import SeqIO
from Bio.Graphics import GenomeDiagram
from Bio.SeqUtils import GC
from Bio import Phylo
from Bio.Align.Applications import MuscleCommandline
from StringIO import StringIO
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.Seq import MutableSeq

from collections import Counter
from itertools import groupby
from operator import itemgetter
import vcf

### (1) Load SNP genotype matrix and Annotation Files

In [4]:
#load isolate annotation file (columns of Genotype Matrix)
isolate_annotation_DF = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_isolate_annotation.pkl')

#load SNP annotation file (rows of Genotype Matrix) with gene annotation information
SNP_annotation_DF = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_SNP_annotation.pkl')

#load Genotypes Matrix
genotypes_array =  np.load('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_matrix.npy')

Columns of Genotype Matrix

In [5]:
isolate_annotation_DF.head()

,lineage_1,lineage_2,lineage_3,lineage_4,lineage_5,lineage_6,lineage_7,lineage_8,lineage_9,lineage_10,lineage_11,isolate_ID,lineage_call,group
0,4,2,1,2,1,1,i3,1,NaN,NaN,NaN,SAMEA3558733,4.2.1.2.1.1.i3.1,4B
1,4,2,1,2,2,1,1,NaN,NaN,NaN,NaN,SAMN03648641,4.2.1.2.2.1.1,4B
2,3,1,1,i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAMN03647419,3.1.1.i1,3
3,4,2,1,2,1,1,i1,NaN,NaN,NaN,NaN,SAMEA3671418,4.2.1.2.1.1.i1,4B
4,1,1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAMN07659096,1.1.1.2,1


In [6]:
np.shape(isolate_annotation_DF)

(31428, 14)

Rows of Genotype Matrix

In [7]:
SNP_annotation_DF.head()

,pos,ref,alt,gene_id,gene_pos,gene_category,gene_name
0,48,C,[T],Rv0001,47.0,Essential,dnaA
1,64,G,[C],Rv0001,63.0,Essential,dnaA
2,67,G,"[A, T]",Rv0001,66.0,Essential,dnaA
3,69,C,[T],Rv0001,68.0,Essential,dnaA
4,71,C,[T],Rv0001,70.0,Essential,dnaA


In [8]:
np.shape(SNP_annotation_DF)

(782565, 7)

Genotype Matrix

In [9]:
genotypes_array

array([[1, 1, 1, ..., 1, 1, 1],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [1, 9, 9, ..., 1, 1, 1],
       [1, 9, 1, ..., 1, 1, 1],
       [1, 9, 1, ..., 1, 1, 1]], dtype=int8)

In [10]:
np.shape(genotypes_array)

(782565, 31428)

### (2) Load in phylogeny created by Luca tree pipeline

We're going to use Biopython's *Phylo* module to load phylogenetic trees created by Luca

In [11]:
#path to tree
phylogeny_path = '/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/phylogenies/tree_output_files/phylogeny_lineage_' + global_lineage + '/tree_lineage_' + global_lineage + '_iqtree_FINAL.treefile' 

#parses and load tree
tree = Phylo.parse(phylogeny_path , 'newick').next() 

#root the tree with the outgroup M. canettii ["Normally you’ll want the outgroup to be a monophyletic group, rather than a single taxon."]
tree.root_with_outgroup({"name":"canettii"})

'''
Can choose to root the tree at the midpoint instead
#root the tree at the midpoint ["Root the tree at the midpoint of the two most distant taxa."]
tree.root_at_midpoint()
'''

#retrieves the terminal branches of the tree
terminal_branches = tree.get_terminals() 

Number of branches on this tree

In [12]:
len(terminal_branches)

8091

Retreive the isolates that were used for this tree

In [13]:
isolate_tags_in_phylogeny = [terminal_branches[i].name for i in range(0 , len(terminal_branches))]

In [14]:
len(isolate_tags_in_phylogeny) #+1 for M. canetti

8091

### (3) Combine *genotypes matrix* and *phylogeny* (subset to intersecting isolate IDs)

#### Find the intersection of isolate tags from Phylogeny & Genotypes Matrix

In [15]:
isolate_tags_to_keep = list( set(isolate_annotation_DF.isolate_ID).intersection( set(isolate_tags_in_phylogeny) ) )

In [16]:
len(isolate_tags_to_keep)

8090

#### Subset *terminal branches* from phylogeny to those included in genotype matrix

In [17]:
isolate_tags_in_phylogeny_and_matrix_ordered = [isolate_tag for isolate_tag in isolate_tags_in_phylogeny if isolate_tag in isolate_tags_to_keep]

In [18]:
isolate_tags_in_phylogeny_and_matrix_ordered[0:5]

['00R0025', 'Peru4307', 'Peru5110', 'SAMEA5569700', 'SAMN06092492']

#### Subset *genotypes matrix* to to only those isolates that are included in phylogeny

In [19]:
isolates_to_keep_filter = [isolate_tag in isolate_tags_to_keep for isolate_tag in isolate_annotation_DF.isolate_ID]

#filter isolate annotation df
isolate_annotation_DF = isolate_annotation_DF[isolates_to_keep_filter]
isolate_annotation_DF.reset_index(drop = True , inplace = True)

#filter genotypes matrix
genotypes_array = genotypes_array[ : , np.array(isolates_to_keep_filter)] 

In [20]:
isolate_annotation_DF.head(n=2)

,lineage_1,lineage_2,lineage_3,lineage_4,lineage_5,lineage_6,lineage_7,lineage_8,lineage_9,lineage_10,lineage_11,isolate_ID,lineage_call,group
0,2,2,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,SAMN05576173,2.2.1.1.1,2
1,2,2,1,1,1,i3,NaN,NaN,NaN,NaN,NaN,SAMN08612836,2.2.1.1.1.i3,2


In [21]:
np.shape(isolate_annotation_DF)

(8090, 14)

In [22]:
SNP_annotation_DF.head(n=2)

,pos,ref,alt,gene_id,gene_pos,gene_category,gene_name
0,48,C,[T],Rv0001,47.0,Essential,dnaA
1,64,G,[C],Rv0001,63.0,Essential,dnaA


In [23]:
np.shape(SNP_annotation_DF)

(782565, 7)

In [24]:
genotypes_array

array([[1, 1, 1, ..., 1, 1, 1],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [1, 9, 1, ..., 1, 1, 1],
       [1, 9, 1, ..., 1, 1, 1],
       [1, 9, 1, ..., 1, 1, 1]], dtype=int8)

In [25]:
np.shape(genotypes_array)

(782565, 8090)

### (4) Drop SNPs where *minor* allele(s) occurs in 0 isolates (SNPs that are monophyletic on the input tree)

Many isolates were dropped in considering only isolates from this lineage. So if there were SNPs in which the alternate allele(s) was present in 2 isolates and 1 was dropped (or both were dropped), then there is only 1 (or 0) isolate(s) with the alternate allele left in the *Genotypes Matrix*. Let's drop all SNPs in which the minor (reference or alternate) allele is present in no isolate(s).

1. For each row of matrix (SNP) we will count the number of **0's**, **1's**, **2's** and **3's**.
1. For each SNP we will take the max( number of 0's , number of 1's , number of 2's , number of 3's ) to get a count of the **major allele**
1. For each SNP we will sum up the total number of isolates with a designated allele (not a 9), to get the count of **non-missing** calls
1. We will then subtract the **count of major alleles** from the **number of non-missing calls** to get the number of isolates with the minor allele(s) for each SNP.
1. We will discard all SNPs for which the minor allele(s) occurs in NO isolates

In [26]:
genotypes_matrix_SNP_0_count = list( np.sum( genotypes_array == 0 , axis = 1 ) ) #A
genotypes_matrix_SNP_1_count = list( np.sum( genotypes_array == 1 , axis = 1 ) ) #C
genotypes_matrix_SNP_2_count = list( np.sum( genotypes_array == 2 , axis = 1 ) ) #G
genotypes_matrix_SNP_3_count = list( np.sum( genotypes_array == 3 , axis = 1 ) ) #T

genotypes_matrix_SNP_non_missing_count = np.sum( genotypes_array != 9 , axis = 1 ) #Non-Missing Data
genotypes_matrix_major_allele_count = np.array( [ max(zero_count , one_count , two_count , three_count) for zero_count , one_count , two_count , three_count in zip(genotypes_matrix_SNP_0_count , genotypes_matrix_SNP_1_count , genotypes_matrix_SNP_2_count , genotypes_matrix_SNP_3_count) ] )

#Number of isolates with the minor allele(s) for each SNP
genotypes_matrix_minor_alleles_count = genotypes_matrix_SNP_non_missing_count - genotypes_matrix_major_allele_count

In [27]:
genotypes_matrix_minor_alleles_count

array([5, 5, 1, ..., 0, 0, 0])

In [28]:
len(genotypes_matrix_minor_alleles_count)

782565

Number of SNPs that we're going to drop because the minor (alternate or reference) allele(s) is present in *no* isolates.

In [29]:
np.sum( np.array( genotypes_matrix_minor_alleles_count ) == 0 )

626756

In [30]:
#boolean list that is 'True' for all SNPs where the minor allele is present in at least 1 isolates
SNPs_to_keep_filter = list( np.array( genotypes_matrix_minor_alleles_count ) > 0 )

SNPs that had at least 1 isolate with minor allele

In [31]:
np.sum(SNPs_to_keep_filter)

155809

In [32]:
#filter SNP annotation DF
SNP_annotation_DF = SNP_annotation_DF[SNPs_to_keep_filter]

#reindex SNP annotation DF
SNP_annotation_DF.reset_index(drop = True , inplace = True)

#filter Genotype Matrix 
genotypes_array = genotypes_array[np.array(SNPs_to_keep_filter) , : ]

In [33]:
np.shape(genotypes_array)

(155809, 8090)

In [34]:
np.shape(SNP_annotation_DF)

(155809, 7)

In [35]:
SNP_annotation_DF.head()

,pos,ref,alt,gene_id,gene_pos,gene_category,gene_name
0,48,C,[T],Rv0001,47.0,Essential,dnaA
1,64,G,[C],Rv0001,63.0,Essential,dnaA
2,67,G,"[A, T]",Rv0001,66.0,Essential,dnaA
3,156,G,[T],Rv0001,155.0,Essential,dnaA
4,157,C,[T],Rv0001,156.0,Essential,dnaA


In [36]:
#output tree specific SNP annotation as CSV
SNP_annotation_DF.to_csv('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/SNP_ancestral_reconstruction/lineage' + global_lineage + '/SNP_annotation.csv')

Number of missing calls in genotypes matrix

In [37]:
np.sum(genotypes_array == 9)

10035450

Proportion of missing calls in genotypes matrix

In [38]:
float( np.sum(genotypes_array == 9) ) / float( (np.shape(genotypes_array)[0] * np.shape(genotypes_array)[1]) ) * 100

0.7961516319135022

### (5) Store a string of genotype characters for each isolate in this global lineage (construct SNP table)

Allele <-> code mapping

In [39]:
allele_code_map_dict = {0:'A' , 1:'C' , 2:'G' , 3:'T' , 9:'-'}

#### Iterate through all of the isolates and store a vector of nucleotides for the SNP concatenate

In [40]:
genotypes_vectors_list = []

for isolate_ID in isolate_annotation_DF.isolate_ID:

    #get the column of genotypes for this specific isolate
    isolate_i = isolate_annotation_DF[isolate_annotation_DF.isolate_ID == isolate_ID].index[0] 

    #get the genotypes for this isolate
    genotypes_for_isolate = genotypes_array[: , isolate_i] 

    #convert the genotypes (coded as numbers) into a string of nucleotides to represent the SNP concatenate for this isolate
    genotypes_string_isolate_i = [allele_code_map_dict[base_call_i] for base_call_i in genotypes_for_isolate]
    
    #append genotype vector for this isolate to the list with all genotype records
    genotypes_vectors_list.append(genotypes_string_isolate_i)

#### Convert list of genotypes vector into a Pandas DataFrame (rows : SNP ref positions / columns : isolate IDs)

In [41]:
#convert to numpy character array
genotypes_SNP_table_array = np.array(genotypes_vectors_list).T

#convert numpy array to Pandas DataFrame
genotypes_SNP_table_DF = pd.DataFrame(data=genotypes_SNP_table_array , index=SNP_annotation_DF.pos , columns=list(isolate_annotation_DF.isolate_ID))

In [42]:
genotypes_SNP_table_DF.head()

,SAMN05576173,SAMN08612836,SAMN06920104,SAMN08612839,SAMN03649023,SAMN03647495,SAMN02586089,SAMN08626029,SAMN05575388,SAMN07766063,...,SAMEA2709814,SAMEA2533971,SAMN05576593,SAMN06055993,SAMEA4441459,SAMN08708740,SAMN12551178,SAMEA1707327,SAMN08626262,SAMN02628527
pos,,,,,,,,,,,,,,,,,,,,,
48,C,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
64,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
67,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
156,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
157,C,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C


In [43]:
np.shape(genotypes_SNP_table_DF)

(155809, 8090)

### (6) Collect the genotypes for *M. canettii*

In [44]:
#make a list that will store the base for each Reference Position for SNP of interest
canettii_genotypes_genotype_vector = []

#path to canettii VCF file
canettii_VCF_path = '/n/data1/hms/dbmi/farhat/lfreschi/mic_assemblies/40-full-analysis/lin-sp-var-10k/results/vcf_canettii/canettii.vcf'

#position-base dictionaries (order of output in Pilon)
base_order = {0:'A' , 1:'C' , 2:'G', 3:'T'}

#load in VCF file if one was found
vcf_reader = vcf.Reader( open(canettii_VCF_path , 'r') )

#there may be 2 (or more) variant calls for a given reference position (common when INDELs are present)
#we will mark these sites as missing/ambiguous
ref_position_genotype_vector_indices_dict = dict([[ref_pos_i, None] for ref_pos_i in genotypes_SNP_table_DF.index]) #keys:Reference Position, values:Genotype Vector Index (if checked) or NONE

genotype_vector_index = 0
#iterate through each Variant Call 
for record in vcf_reader:
    
    #check to see if one of SNP positions of interest 
    if (record.POS in genotypes_SNP_table_DF.index):
        
        #check to see if SNP position hasn't already been counted (multiple variant calls for this ref pos)
        if ref_position_genotype_vector_indices_dict[record.POS] == None:
            
            good_quality_call = False
    
            #check to see if variant is SNP (length of Reference Allele is 1 base and there's only 1 alternate allele and length of alternate allele is 1 base is alternate allele exists)
            if (len(record.REF) == 1) and (len(record.ALT) == 1) and ( (record.ALT[0] == None) or (len(str(record.ALT[0])) == 1) ):

                #check to see if the call is a PASS by Pilon
                if record.FILTER == []:

                    ##### Retrieve Relevant information for filtering quality of Base Call #####
                    # Mean Base Quality @ locus
                    BQ = record.INFO['BQ']
                    # Mean Mapping Quality @ locus
                    MQ = record.INFO['MQ']
                    # Number of Reads w/ Deletion 
                    DC = record.INFO['DC']
                    # Number of Reads w/ Insertion
                    IC = record.INFO['IC']
                    # Depth of Valid Reads in Pileup
                    VD = record.INFO['DP']

                    ### Filtering Criteria
                    #---> Mean Base Quality > 20
                    #---> Mean Mapping Quality > 30
                    #---> No Reads Supporting Insertions
                    #---> No Reads Supporting Deletions
                    #---> Number of High Quality Reads >= 20
                    if (BQ > 20) and (MQ > 30) and (DC == 0) and (IC == 0) and (VD >= 20): 

                        # Quality-Percentage - Percentage of As, Cs, Gs, Ts weighted by Q & MQ at locus
                        QP = record.INFO['QP']

                        if (sum(np.array(QP) >= 75) == 1): #1 Allele is supported by at least 75% of the reads, store information for this Base Call

                            good_quality_call = True

                            #create a Boolean list for the Bases that have at least 75% of support from reads
                            supported_bases_boolean = list( np.array( QP ) >= 75 )

                            #position of Base [A,C,G,T] with at least 90% of support from reads
                            supported_bases_i = [i for i,x in enumerate(supported_bases_boolean) if x == True] #returns list of length 
                            supported_base = base_order[supported_bases_i[0]]

                            #store a 'A', 'C', 'T', or 'G' in Genotype Vector (Reference Position Index) referencing the Base with at least 75% support from the reads
                            canettii_genotypes_genotype_vector.append(supported_base)

                            ref_position_genotype_vector_indices_dict[record.POS] = genotype_vector_index
                            genotype_vector_index += 1

            #call at this Reference Position did not meet filters for SNP calls
            if good_quality_call == False:

                canettii_genotypes_genotype_vector.append('-')

                ref_position_genotype_vector_indices_dict[record.POS] = genotype_vector_index
                genotype_vector_index += 1
            
            
        #if multiple variants for this Ref Position exists, call variant as missing/ambiguous
        elif ref_position_genotype_vector_indices_dict[record.POS] != None:

            canettii_genotypes_genotype_vector[ref_position_genotype_vector_indices_dict[record.POS]] = '-'

Append genotype vector for *canettii* to SNP table DataFrame

In [45]:
np.shape(genotypes_SNP_table_DF)

(155809, 8090)

In [46]:
genotypes_SNP_table_DF.loc[: , 'canettii'] = canettii_genotypes_genotype_vector

In [47]:
np.shape(genotypes_SNP_table_DF)

(155809, 8091)

Output SNP table Dataframe as a CSV file

In [48]:
genotypes_SNP_table_DF.to_csv('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/SNP_ancestral_reconstruction/lineage' + global_lineage + '/SNP_table.csv')

####################################################################################################################################################################################

## [3] This section runs SNPPar with the (SNP table) and (IQTree generated phylogenies) for Lineages 1, 2, 3, 4A, 4B, 4C, 5 & 6

####################################################################################################################################################################################

### Run this for global lineages 1, 2, 3, 4A, 4B, 4C, 5 & 6

In [49]:
from slurmpy import Slurm
import os

In [50]:
global_lineage = '2'
partition = 'priority'
runtime_limit = '21-0:00:00'
sorting = 'intermediate'

In [51]:
#### prepare file and directory paths ####

#input file paths
tree_path = '/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/phylogenies/tree_output_files/phylogeny_lineage_' + global_lineage + '/tree_lineage_' + global_lineage + '_iqtree_FINAL_no_bootstrap_vals.treefile'
SNP_table_file_path = '/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/SNP_ancestral_reconstruction/lineage' + global_lineage + '/SNP_table.csv'
genbank_file_path = '/home/rv76/Farhat_Lab/Reference_Seqs/H37Rv/NC_000962_3.gbk'

#output directories
out_dir = '/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/SNP_ancestral_reconstruction/lineage' + global_lineage + '/SNPPar_output'
SLURM_log_dir = '/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/SNP_ancestral_reconstruction/O2_SLURM_logs'


#### construct job to submit to O2 ####

#store all commands in a list
commands_list = []

#load virtual environment for SNPPar
commands_list.append( 'set +eu' )
commands_list.append( 'source activate SNPPar_virtualenv' )
commands_list.append( 'set -eu' )

#run SNPPar 
commands_list.append( 'snppar --snptable {0} --tree {1} --genbank {2} --sorting {3} --directory {4} --prefix {5} --no_all_calls --no_homoplasic'.format(SNP_table_file_path, tree_path, genbank_file_path, sorting, out_dir, 'L'+global_lineage) )

#### SUBMIT the job to O2 ####

#append all commands in a single string to be submitted as a job
anc_recon_job = ''
for command_i in commands_list:
    anc_recon_job = anc_recon_job + '\n' + command_i
    
#directory where you want output + error files
os.chdir(SLURM_log_dir)

job_name = 'SNPPar_L' + global_lineage

#submit SNPPar job via SLURM
s = Slurm(job_name , {'partition':partition , 'N':'1' , 't':runtime_limit , 'mem':'128G' , 'mail-type':'ALL' , 'mail-user':'roger_vargas@g.harvard.edu'})

#submits the job
job_id = s.run(anc_recon_job)

print job_name  + ' : ' +  str(job_id)

SNPPar_L2 : 13896275


submitted: Submitted batch job 13896275


####################################################################################################################################################################################

## [4] This section parses SNPPar output for Lineages 1, 2, 3, 4A, 4B, 4C, 5, 6 & ensures all mutations events match up with the inferred and terminal sequences

####################################################################################################################################################################################

In [2]:
from collections import Counter
import pandas as pd
import numpy as np

import Bio
from Bio.Alphabet import IUPAC
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import pairwise2
from Bio import SeqIO
from Bio.Graphics import GenomeDiagram
from Bio.SeqUtils import GC
from Bio import Phylo
from Bio.Align.Applications import MuscleCommandline
from StringIO import StringIO
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.Seq import MutableSeq
from Bio.Alphabet import generic_dna

#### Function that checks each mutation reported in the mutation events table (output from SNPPar) against the inferred sequences at the nodes of the phylogeny & the isolate sequences

In [3]:
def mutation_events_check_against_seqs(global_lineage):

    #Load mutation events DataFrame
    mutation_events_DF = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/SNP_ancestral_reconstruction/lineage{0}/SNPPar_output/L{0}all_mutation_events.tsv'.format(global_lineage) , sep = '\t') 

    #Load inferred sequences at the nodes (inferred)
    seqs_dict = SeqIO.to_dict(SeqIO.parse("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/SNP_ancestral_reconstruction/lineage{0}/SNPPar_output/L{0}node_sequences.fasta".format(global_lineage), "fasta"))

    #Load the Reference Position SNP annotation from the SNP table that was used as input
    SNP_table = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/SNP_ancestral_reconstruction/lineage{0}/SNP_table.csv'.format(global_lineage) , sep = ',')

    #convert columns of SNP table into a SeqRecord for each isolate (terminal)
    for isolate_i in SNP_table.columns[1:]:

        isolate_i_seq = SNP_table.loc[: , isolate_i]
        isolate_i_seq[isolate_i_seq == '-'] = 'N'
        isolate_i_seq = ''.join(isolate_i_seq)
        isolate_i_seq_record = SeqRecord(Seq(isolate_i_seq , generic_dna) , id = isolate_i)

        #add seq record to dict
        seqs_dict[isolate_i] = isolate_i_seq_record

    #create a series from the SNP table that maps each SNP concatenate index to H37Rv Reference Position
    SNP_table.loc[: , 'index'] = SNP_table.index
    SNP_pos_index_series = SNP_table.set_index('pos')
    SNP_pos_index_series = SNP_pos_index_series.loc[: , 'index']

    #iterate through each mutation in events table and check against fasta seqs
    mutations_checked_against_seqs_filter = []

    for mutation_event_i in mutation_events_DF.index:

        mutation_event_i_info = mutation_events_DF.loc[mutation_event_i , ['Position' , 'Ancestor_Node' , 'Derived_Node' , 'Ancestor_Call' , 'Derived_Call']]

        #get the Ancestor Call & Derived Call from the inferred/terminal sequences
        Anc_call_from_seqs = seqs_dict[mutation_event_i_info.Ancestor_Node].seq[SNP_pos_index_series[mutation_event_i_info.Position]]
        Der_call_from_seqs = seqs_dict[mutation_event_i_info.Derived_Node].seq[SNP_pos_index_series[mutation_event_i_info.Position]]

        #check calls from sequences against what was reported in the mutation table
        if (mutation_event_i_info.Ancestor_Call == Anc_call_from_seqs) and (mutation_event_i_info.Derived_Call == Der_call_from_seqs):

            mutations_checked_against_seqs_filter.append(True)

        else:

            mutations_checked_against_seqs_filter.append(False)

    mutations_checked_against_seqs_filter = np.array(mutations_checked_against_seqs_filter)

    print 'There were ' + str(len(mutations_checked_against_seqs_filter)) + ' reported events in the mutation table between the inferred node & isolate sequences.'
    print 'There were ' + str(sum(mutations_checked_against_seqs_filter == False)) + ' reported events in the mutation table that were not located between the inferred node & isolate sequences.'
    
    '''
    #This block of code outputs the reported mutation events in the mutation table that were not located between the inferred node & isolates sequences as a TSV file.
    #collect mutation events that were not found among the seqs
    bad_reported_mutations_filter = np.array([not good_report for good_report in mutations_checked_against_seqs_filter])
    bad_mutation_events_DF = mutation_events_DF[bad_reported_mutations_filter]
    bad_mutation_events_DF.set_index('Position' , drop = True , inplace = True)
    bad_mutation_events_DF.to_csv('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/SNP_ancestral_reconstruction/lineage{0}/SNPPar_output/L{0}false_reported_mutation_events_checked_against_seqs.tsv'.format(global_lineage) , sep = '\t')
    '''
    
    #filter out the mutation events that were not found among the inferred / isolate sequences and save
    mutation_events_DF = mutation_events_DF[mutations_checked_against_seqs_filter]
    mutation_events_DF.set_index('Position' , drop = True , inplace = True)
    ## mutation_events_DF.to_csv('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/SNP_ancestral_reconstruction/lineage{0}/SNPPar_output/L{0}all_mutation_events_checked_against_seqs.tsv'.format(global_lineage) , sep = '\t')

In [4]:
mutation_events_check_against_seqs('1')

/home/rv76/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2824: DtypeWarning: Columns (7,8,9,16,17,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/rv76/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


There were 204695 reported events in the mutation table between the inferred node & isolate sequences.
There were 447 reported events in the mutation table that were not located between the inferred node & isolate sequences.


In [4]:
mutation_events_check_against_seqs('2p')

/home/rv76/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2824: DtypeWarning: Columns (7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/rv76/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


There were 2471 reported events in the mutation table that were not located between the inferred node & isolate sequences.


In [5]:
mutation_events_check_against_seqs('2')

/home/rv76/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2824: DtypeWarning: Columns (7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/rv76/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


There were 193418 reported events in the mutation table between the inferred node & isolate sequences.
There were 2472 reported events in the mutation table that were not located between the inferred node & isolate sequences.


In [6]:
mutation_events_check_against_seqs('3')

/home/rv76/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


There were 138252 reported events in the mutation table between the inferred node & isolate sequences.
There were 392 reported events in the mutation table that were not located between the inferred node & isolate sequences.


In [7]:
mutation_events_check_against_seqs('4A')

/home/rv76/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


There were 156743 reported events in the mutation table between the inferred node & isolate sequences.
There were 839 reported events in the mutation table that were not located between the inferred node & isolate sequences.


In [8]:
mutation_events_check_against_seqs('4B')

/home/rv76/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


There were 146916 reported events in the mutation table between the inferred node & isolate sequences.
There were 1177 reported events in the mutation table that were not located between the inferred node & isolate sequences.


In [9]:
mutation_events_check_against_seqs('4C')

/home/rv76/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


There were 157183 reported events in the mutation table between the inferred node & isolate sequences.
There were 559 reported events in the mutation table that were not located between the inferred node & isolate sequences.


In [10]:
mutation_events_check_against_seqs('5')

/home/rv76/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


There were 8920 reported events in the mutation table between the inferred node & isolate sequences.
There were 2 reported events in the mutation table that were not located between the inferred node & isolate sequences.


In [11]:
mutation_events_check_against_seqs('6')

/home/rv76/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


There were 11456 reported events in the mutation table between the inferred node & isolate sequences.
There were 3 reported events in the mutation table that were not located between the inferred node & isolate sequences.
